# Working in Impala
## Step 1 is to connect to the correct host
Using impyla seems to mimics the impala-shell (CLI) pretty closely.

Some important points to keep in mind/equivalencies

* `impala-shell -i host:port` is equivalent to `connect(host=hostname,port=portnumber)`
* `cursor = connect(...).cursor()` yields access to the CLI via:
    - `cursor.execute(query)` followed by
    - `cursor.fetchall()` to return the results
* note that the queries don't have to end in a semicolon.    


In [1]:
# connect to the correct host
myhostname = 'ac00h1pdata02.opr.statefarm.org' #hostname of G-building
myhostname = '
portnum = 21050 # default port number ??

In [14]:
# connect to the correct host
myhostname = 'da74wbdn01.opr.statefarm.org' #hostname of PHDac00h1pdata02.opr.statefarm.org' #hostname of G-building
#myhostname = '' 
#you may use any of the datanodes from da74wbdn01 to da74wbdn27
portnum = 21050 # default port number ??

In [15]:
from impala.dbapi import connect
conn = connect(host=myhostname, port=portnum,use_kerberos=True)
cur = conn.cursor()

In [16]:
cur.execute('SHOW databases')

In [17]:
cur.fetchall()

[('default',), ('lapsecan',), ('vehrepat',)]

## Some utility functions 
1. list databases
    * type `cursor.execute("use dbasename")` to change to database
2. list tables
3. count lines in a table

In [5]:
# list databases
def listDatabases(cursor):
    """
    input: the cursor connection input
    output: list of tables in active database
    """
    cursor.execute('SHOW databases')
    dbList = cursor.fetchall()
    return dbList

# list tables
def listTables(cursor):
    """
    input: the cursor connection input
    output: list of tables in active database
    """
    cursor.execute('SHOW tables')
    tblList = cursor.fetchall()
    return tblList
# count the number of lines in a given table
def countLines(cursor,tablename):
    qstring = "'select count(*) from "+tablename+"'"
    return cursor.execute(qstring)

In [12]:
cur.close()

In [18]:
# list the databases
listDatabases(cur)

[('default',), ('lapsecan',), ('vehrepat',)]

In [19]:
# change to desired database
cur.execute("use vehrepat")

In [20]:
myTables=listTables(cur)
print len(myTables)
myTables

2


[('p3533eeb_detl',), ('y1753caaestparty',)]

In [24]:
from impala.util import as_pandas
#cur.execute('SELECT * FROM y1753caale')
#cur.execute('select * FROM p3533eeb_detl')
cur.execute('select * FROM y1753caaestparty')
le = as_pandas(cur)
print len(le)


0


In [25]:
le.head()

,est_party_dim_id,last_nm,first_nm,extnl_party_id,addr_line_one_txt,addr_line_two_txt,city_nm,st_prvnc_cd,zip_cd,cntry_cd,...,extnl_vndr_id_type_cd,clm_id,data_cntxt_cd,fdw_rplc_ind,fdw_insrt_tstmp,fdw_rplc_tstmp,est_ver_num,src_insrt_tstmp,los_est_busn_id,los_est_dim_id


In [ ]:
#sq1 = "'select * from "+myTables[1][0]+"'"
#cur.execute(sq1)
cur.execute('select * from y1753caadetl')
detl = as_pandas(cur)

In [ ]:
print shape(detl)

In [ ]:
len(le.groupby('los_est_dim_id'))

In [ ]:
le.los_est_dim_id.head()

In [ ]:
ledi0 = 85051447
sum(detl['los_est_dim_id']==ledi0)
#detl[['los_est_dim_id','est_ver_num','prt_clas_cd']]

In [ ]:
detl[detl['los_est_dim_id']==ledi0][['est_ver_num','oem_prt_num','line_desc_txt','prt_clas_cd','prt_qty_cnt','price_amt','ver_num',]]

In [ ]:
print len(le[le['los_est_dim_id']==ledi0])
le[le['los_est_dim_id']==ledi0][['vndr_veh_cd','los_tstmp','prmry_poi_cd','scdy_poi_cd','fdw_rplc_tstmp']]

In [ ]:
le.columns.values

In [ ]:
len(detl.prt_clas_cd.unique())

In [ ]:
y17533caaPRTcounts = detl.prt_clas_cd.value_counts()
y17533caaPRTcounts.tail(-1).hist(color='steelblue',bins=50)

In [ ]:
detl[detl.prt_clas_cd == '88JHR']['line_desc_txt']

In [ ]:
le[le.los_est_dim_id == ledi0]['vndr_cd']

In [14]:
cur.execute("CREATE TABLE page_view(viewTime INT, userid BIGINT,\
                page_url STRING, referrer_url STRING,\
                ip STRING COMMENT 'IP Address of the User')\
COMMENT 'This is the page view table'\
PARTITIONED BY(dt STRING, country STRING)\
CLUSTERED BY(userid) SORTED BY(viewTime) INTO 32 BUCKETS\
ROW FORMAT DELIMITED\
        FIELDS TERMINATED BY '1'\
        COLLECTION ITEMS TERMINATED BY '2'\
        MAP KEYS TERMINATED BY '3'\
STORED AS SEQUENCEFILE;")

RPCError: RPC status error: TExecuteStatementResp: TStatus(errorCode=None, errorMessage="AnalysisException: Syntax error in line 1:\n...is is the page view table'PARTITIONED BY(dt STRING, co...\n                             ^\nEncountered: PARTITIONED\nExpected: AS, LOCATION, ROW, STORED, TBLPROPERTIES, WITH, COMMA\n\nCAUSED BY: Exception: Syntax error", sqlState='HY000', infoMessages=None, statusCode=3)

In [ ]:
from pandas import *
from StringIO import StringIO

s = "sshpass -f myfilewithpassword ssh myusername@myhostname \"hive -S -e \\\"" \
"set hive.cli.print.header=true;" \
"SELECT * from mytable;\\\"\""

t = !$s
df = read_csv(StringIO(t.n), sep='\t')

In [ ]:
from pandas import *
from StringIO import StringIO

s = """
sshpass -f myfilewithpassword ssh myusername@myhostname \"
hive -S -e \\\"
set hive.cli.print.header=true;
SELECT * from mytable;
\\\"\"
"""

t = !$s
df = read_csv(StringIO(t.n), sep='\t'

In [16]:
hdfsfiles = !hadoop fs -ls 